In [13]:
from langgraph.graph import StateGraph, END, START
from langgraph.types import Command, interrupt
from typing import TypedDict, List
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
class State(TypedDict):
    value: str
    
def nodeA(state: State):
    print('A')
    return Command(
        goto='nodeB',
        update={'value':state['value']+'A'}
    )

def nodeB(state: State):
    print('B')
    human_response = interrupt('Do you want to go to C or D?')
    print("Human Review Values: ", human_response)

    if human_response == 'C':
        
        return Command(
        goto = 'nodeC',
        update={'value':state['value']+'B'}
        )

    elif human_response == 'D':
        return Command(

            goto='nodeD',
            update = {'value':state['value']+'B'}
        )

In [14]:
def nodeC(state: State): 
    print("nodeC")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "c"
        }
    )

def nodeD(state: State): 
    print("nodeD")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "d"
        }
)

In [15]:
graph = StateGraph(State)

graph.add_node("nodeA", nodeA)
graph.add_node("nodeB", nodeB)
graph.add_node("nodeC", nodeC)
graph.add_node("nodeD", nodeD)

graph.set_entry_point("nodeA") 
app = graph.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "1"}}

initialState = {
    "value": ""
}

result1 = app.invoke(initialState, config, stream_mode="updates")
result1

A
B


[{'nodeA': {'value': 'A'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D?', resumable=True, ns=['nodeB:c26b5567-4b1d-c02e-7b44-0b733742a574']),)}]

In [16]:
print(app.get_state(config).next)
#resuming the graph again##
result2 = app.invoke(Command(resume="C"), config=config, stream_mode="updates")
result2

('nodeB',)
B
Human Review Values:  C
nodeC


[{'nodeB': {'value': 'AB'}}, {'nodeC': {'value': 'ABc'}}]